## AutoReload

In [1]:
%load_ext autoreload
%autoreload 2

## 0. parameter store

In [2]:
import boto3
from utils.ssm import parameter_store

In [3]:
strRegionName=boto3.Session().region_name
pm = parameter_store(strRegionName)

## 1. Set DynamoDB

In [4]:
from utils.ddb import ddb_constructor

In [5]:
ddb_construction = ddb_constructor(region_name=pm.get_params(key="DAT-REGION"))

### 1.1 Create table: assignmnet

In [6]:
ddb_construction.create_table(
    AttributeDefinitions=[
        {'AttributeName': 'user_id', 'AttributeType': 'S'}, # 'S'|'N'|'B' 
    ],
    TableName='DAT-assignment',
    KeySchema=[
        {'AttributeName': 'user_id', 'KeyType': 'HASH'}, # 'HASH'(partition)|'RANGE'(sort)
    ],
    BillingMode='PAY_PER_REQUEST',# 'PROVISIONED'|'PAY_PER_REQUEST'('ProvisionedThroughput' requried),  
)

dynamodb.Table(name='DAT-assignment') was created!!


### 3.2 Table create: metric

In [7]:
ddb_construction.create_table(
    AttributeDefinitions=[
        {'AttributeName': 'variant_name', 'AttributeType': 'S'}, # 'S'|'N'|'B' 
    ],
    TableName='DAT-metric',
    KeySchema=[
        {'AttributeName': 'variant_name', 'KeyType': 'HASH'}, # 'HASH'(partition)|'RANGE'(sort)
    ],
    BillingMode='PAY_PER_REQUEST',# 'PROVISIONED'|'PAY_PER_REQUEST'('ProvisionedThroughput' requried),
)

dynamodb.Table(name='DAT-metric') was created!!


### 3.3 Table create: helpful_reviews

In [8]:
ddb_construction.create_table(    
    AttributeDefinitions=[
        {'AttributeName': 'product_id', 'AttributeType': 'S'}, # 'S'|'N'|'B' 
        {'AttributeName': 'variant_name', 'AttributeType': 'S'}, # 'S'|'N'|'B' 
    ],
    TableName='DAT-helpful-reviews',
    KeySchema=[
        {'AttributeName': 'product_id', 'KeyType': 'HASH'}, # 'HASH'(partition)|'RANGE'(sort)
        {'AttributeName': 'variant_name', 'KeyType': 'RANGE'} # In the sort key, only "RANGE" of Key type is available 
    ],
    BillingMode='PAY_PER_REQUEST',# 'PROVISIONED'|'PAY_PER_REQUEST'('ProvisionedThroughput' requried),
)

dynamodb.Table(name='DAT-helpful-reviews') was created!!


### 3.4 Table create: metric-visualization

In [9]:
ddb_construction.create_table(
    AttributeDefinitions=[
        {'AttributeName': 'time_stamp', 'AttributeType': 'S'}, # 'S'|'N'|'B'
        {'AttributeName': 'variant_name', 'AttributeType': 'S'}, # 'S'|'N'|'B' 
    ],
    TableName='DAT-metric-visualization',
    KeySchema=[
        {'AttributeName': 'time_stamp', 'KeyType': 'HASH'}, # 'HASH'(partition)|'RANGE'(sort)
        {'AttributeName': 'variant_name', 'KeyType': 'RANGE'}, # In the sort key, only "RANGE" of Key type is available 
    ],
    BillingMode='PAY_PER_REQUEST',# 'PROVISIONED'|'PAY_PER_REQUEST'('ProvisionedThroughput' requried),
    #ProvisionedThroughput={
    #    'ReadCapacityUnits': 123,
    #    'WriteCapacityUnits': 123
    #},
)

dynamodb.Table(name='DAT-metric-visualization') was created!!


## 2. Set Labmda

In [10]:
import shutil
import zipfile
import tempfile
from utils.lambda_func import lambda_handler

In [11]:
lam_handler = lambda_handler(region_name=pm.get_params(key="DAT-REGION"))

In [12]:
strLambdaRoleArn = pm.get_params(key="DAT-LAMBDA-ROLE-ARN")
strLambdaFuncName = "DAT-Lambda-MAB"
strLambdaSrcDir = "./ab-test/lambda"
strLambdaHandler = "ab_test_with_mab.lambda_handler"

In [18]:
with tempfile.TemporaryDirectory() as tempDirPath:
    
    lambda_archive_path = shutil.make_archive(
        base_name=tempDirPath,
        format="zip",
        root_dir=strLambdaSrcDir,
    )
    
    with open(lambda_archive_path, 'rb') as f: zipped_code = f.read()
    
    strLambdaArn = lam_handler.create_function(
        Code=dict(ZipFile=zipped_code),
        Description='Process A/B Testing with MAB.',
        Environment={
           'Variables': {
               'DDB_ASSIGNMENT': "DAT-assignment",
               'DDB_METRIC': "DAT-metric",
               'DDB_METRIC_VISUALIZATION': "DAT-metric-visualization",
               'DDB_HELPFUL_REVIEWS': "DAT-helpful-reviews",
               'REGION':pm.get_params(key="DAT-REGION"),
               'MODEL_ENDPOINT':pm.get_params(key="DAT-MODEL-ENDPIONT-NAME"),
               'WARM_CNT':"150",
           },
        },
        FunctionName=strLambdaFuncName,
        Handler=strLambdaHandler,
        Publish=True,
        Role=strLambdaRoleArn,
        Runtime='python3.8',
    )
    
print (f"LambdaArn: {strLambdaArn}")

== CREATE ROLE ==
Argments for lambda below:

{'Architectures': ['x86_64'],
 'CodeSha256': 'o/k9oxtnu/R6ve086hQajCrOGJ/AZIOO0jzp7HKVq9I=',
 'CodeSize': 9488,
 'Description': 'Process A/B Testing with MAB.',
 'Environment': {'Variables': {'DDB_ASSIGNMENT': 'DAT-assignment',
                               'DDB_HELPFUL_REVIEWS': 'DAT-helpful-reviews',
                               'DDB_METRIC': 'DAT-metric',
                               'DDB_METRIC_VISUALIZATION': 'DAT-metric-visualization',
                               'MODEL_ENDPOINT': 'helpfulness-detector-endpoint-2022-12-02-06-58-09',
                               'REGION': 'ap-northeast-2',
                               'WARM_CNT': '150'}},
 'EphemeralStorage': {'Size': 512},
 'FunctionArn': 'arn:aws:lambda:ap-northeast-2:419974056037:function:DAT-Lambda-MAB',
 'FunctionName': 'DAT-Lambda-MAB',
 'Handler': 'ab_test_with_mab.lambda_handler',
 'LastModified': '2022-12-02T07:05:51.412+0000',
 'MemorySize': 128,
 'PackageType': '

## 3. Set API-Gateway with Lambda

In [19]:
from utils.api_gateway import api_gateway_handler

In [20]:
strAccountId = pm.get_params(key="DAT-ACCOUNT-ID")
strApiName = "DAT-api-gateway"
strApiDesc = "api-gatway for A/B Testing with MAB"
strApiBasePath = "dat-api"
strApiStage = "dev"
strLambdaFuncArn = strLambdaArn
apig = api_gateway_handler(pm.get_params(key="DAT-REGION"))

In [21]:
strApiUrl = apig.create_rest_api_with_lambda(
    api_name=strApiName,
    api_description=strApiDesc,
    api_base_path=strApiBasePath,
    api_stage=strApiStage,
    account_id=strAccountId,
    lambda_function_arn=strLambdaFuncArn
)
pm.put_params(key="DAT-API-URL", value=strApiUrl.encode('utf-8'), overwrite=True)

Creating Amazon API Gateway REST API DAT-api-gateway...
so arn:aws:execute-api:ap-northeast-2:419974056037:m07tmppemg/*/*/dat-api
REST API created, URL is :
	https://m07tmppemg.execute-api.ap-northeast-2.amazonaws.com/dev/dat-api
Sleeping for a couple seconds to give AWS time to prepare...


'Store suceess'

In [22]:
pm.put_params(key="DAT-API-URL", value=strApiUrl, overwrite=True)


Error parsing parameter '--value': Unable to retrieve https://m07tmppemg.execute-api.ap-northeast-2.amazonaws.com/dev/dat-api: received non 200 status code of 403


'Error'